In [20]:
%%capture
!pip install llama-index==0.12.0 llama-index-embeddings-openai==0.1.9 qdrant-client==1.9.1 llama-index-vector-stores-qdrant==0.2.8 llama-index-llms-cohere==0.2.0

In [16]:
!pip install python-dotenv

In [10]:
!pip install llama-index-vector-stores-deeplake

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 21.7 MB/s eta 0:00:00
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.10.68.post1
    Uninstalling llama-index-core-0.10.68.post1:
      Successfully uninstalled llama-index-core-0.10.68.post1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.10.37 requires llama-index-core<0.11.0,>=0.10.35, but you have llama-index-core 0.12.21 which is incompatible.
llama-index-agent-openai 0.2.9 requires llama-index-core<0.11.0,>=0.10.41, but you have llama-index-core 0.12.21 which is incompatible.
llama-index-cli 0.1.13 requires llama-index-core<0.11.0,>=0.10.11.post1, but you have llama-index-core 0.12.21 which is incompatible.
llama-index-embeddings-cohere 0.1.8 requires llama-index-core<0.11.0,>=0.10.1, but you have llama-index-core 0.12.21 which is incompatible.
llama-index-embeddings-openai 0.1.9 requires llama-index-core<0.11.0,>=0.10.1, but you have llama-index-core 0.12.21 which is incompatible.
llama-index-indices-managed-llama-cloud 0.1.6 requires llama-index-core<0.11.0,>=0.10.0, but you have llama-index-core 0.12.21 which is incompatible.


In [7]:
import os
from dotenv import load_dotenv


import nest_asyncio

nest_asyncio.apply()
load_dotenv()


True

In [8]:
API_KEY_CO= os.getenv("API_KEY_COHERE")
API_KEY_OPEN_AI = os.getenv("API_KEY_OPEN_AI")
API_KEY_QDRANT = os.getenv("API_KEY_QDRANT")
QDRANT_CLIENT = os.getenv("QDRANT_CLIENT")


In [9]:
print(QDRANT_CLIENT)

https://123aa9b9-5700-4be7-8a59-a4918adaea1f.eu-west-2-0.aws.cloud.qdrant.io


In [12]:
from llama_index.core.settings import Settings
from llama_index.llms.cohere import Cohere
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = Cohere(model="command-r-plus", api_key=API_KEY_CO)

Settings.embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\anteb\anaconda3\Lib\site-
[nltk_data]     packages\llama_index\core\_static/nltk_cache...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


In [13]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files = [r"C:\Users\anteb\OneDrive\Desktop\Griffith\Thesis\RAG\AGENTIC RETRIEVAL-AUGMENTED GENERATION A SURVEY ON AGENTIC RAG.pdf"],
    filename_as_id=True).load_data()

In [19]:
!pip install deeplake==3.9.12


     ---------------------------------------- 0.0/606.9 kB ? eta -:--:--
     ------------------------------------- 606.9/606.9 kB 11.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ----------------------------------- ---- 2.4/2.6 MB 33.8 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 7.2 MB/s eta 0:00:00
  Created wheel for deeplake: filename=deeplake-3.9.12-py3-none-any.whl size=729581 sha256=10f15be11d0e9b947a24a9d6efd54f589b8c515ee9f1ae3bc3a8bb8dac195458
  Stored in directory: c:\users\anteb\appdata\local\pip\cache\wheels\92\67\0f\3ada0b15e577752bdd57d7d801ec8e37f4024a1707978ed0a1
Successful

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastembed 0.6.0 requires pillow<12.0.0,>=10.3.0, but you have pillow 10.2.0 which is incompatible.
llama-index-embeddings-fastembed 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llama-index-embeddings-langchain 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llama-index-llms-mistralai 0.3.3 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llama-index-vector-stores-deeplake 0.3.2 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.10.68.post1 which is incompatible.


In [25]:
from qdrant_client import QdrantClient
from llama_index.core import VectorStoreIndex, StorageContext
# Delaying this import
# from llama_index.vector_stores.deeplake import DeepLakeVectorStore
from llama_index.core.ingestion import IngestionCache, IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter

# Move this import after the other imports, delaying it until necessary
from llama_index.vector_stores.deeplake import DeepLakeVectorStore


# construct vector store and customize storage context
vector_store_db = DeepLakeVectorStore(dataset_path=r"C:\Users\anteb\Desktop\Courses")
storage_context = StorageContext.from_defaults(vector_store=vector_store_db)
# Load documents and build index
index_1 = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)


## reload an existing one
index_1 = VectorStoreIndex.from_vector_store(vector_store=vector_store_db)

# client = QdrantClient(
#     url=QDRANT_CLIENT,
#     api_key=API_KEY_QDRANT,
# )
# vector_store = QdrantVectorStore(
#     client=client,
#     collection_name="new_collection",)
#
# ingest_cache = IngestionCache(
#     collection="new_collection",
# )
#
# # create pipeline with transformations
# pipeline = IngestionPipeline(
#     transformations=[
#         TokenTextSplitter(chunk_size=256, chunk_overlap=16),
#         Settings.embed_model
#     ],
#     docstore=SimpleDocumentStore(),
#     vector_store=vector_store,
#     cache=ingest_cache,
# )
#
# # run the pipeline
# nodes = pipeline.run(documents = documents)

AttributeError: partially initialized module 'deeplake' has no attribute 'core' (most likely due to a circular import)